In [ ]:

import numpy as np
import random
import math


In [ ]:
#  Simulated Annealing
# TODO: ADD ENDING WHERE WE START


# Distance matrix (10 stations with random distances)
# distance_matrix = np.random.randint(1, 100, size=(10, 10))
# np.fill_diagonal(distance_matrix, 0)

class TSP:
    def __init__(self, num_stations, min_max_distance=(1, 100)):
        self.distance_matrix = self.generate_distance_matrix(num_stations, min_max_distance)
        self.current_route = np.random.permutation(num_stations) # Random intial route

        self.distances = [] # Tuples of (route, total distance of each route)


    def generate_distance_matrix(self, num_stations, min_max_distance):
        """
        Generate a random distance matrix for the given number of stations.
        distance_matrix[i, j] represents the distance from station i to station j
        """

        distance_matrix = np.random.randint(*min_max_distance, size=(num_stations, num_stations))
        np.fill_diagonal(distance_matrix, 0)
        return distance_matrix

    def swap_two_stations(self, route):
        """
        Swap two stations in the route to create a new route
        Works since each stop along the route is a station with an already implemented from - to
        """

        new_route = route.copy()
        i, j = random.sample(range(len(route)), 2) # Sample two stops along the route
        new_route[i], new_route[j] = new_route[j], new_route[i]

        # self.current_route = new_route
        return new_route

    def total_distance(self, route, end_at_start=False):
        """
        Calculate the total distance of the given route
        For each stop, calculate the distance from previous stop to that stop
        """
        dist = 0
        for i in range(len(route)):
            dist += self.distance_matrix[route[i-1], route[i]]
        
        if end_at_start:
            dist += self.distance_matrix[route[-1], route[0]]
        
        return dist

    def simulated_annealing(self, max_iterations):
        """Perform Simulated Annealing to solve the TSP."""
        # Initialize the current state
        # num_stations = distance_matrix.shape[0]
        # current_route = list(range(num_stations))
        # random.shuffle(current_route)

        # TODO: REMOVE self.current route and just use as regular argumnet, no reason to have it frr
        current_route = self.current_route
        current_distance = self.total_distance(self.current_route)

        best_route = self.current_route
        best_distance = current_distance
        
        # temp = initial_temp # CAn perhaps setp initial temp?
        
        for k in range(max_iterations):
            temp = 1 / math.sqrt(1 + k)
            
            # NEed this? Does temp ever go that fucking low?
            # if temp < 1e-8:
            #     break
            self.distances.append([k, self.current_route, current_distance])

            new_route = self.swap_two_stations(current_route)
            new_distance = self.total_distance(new_route, end_at_start=True)
            
            if new_distance < current_distance:
                current_route = new_route
                current_distance = new_distance
            else:
                acceptance_prob = math.exp(-(new_distance - current_distance) / temp)
                if random.random() < acceptance_prob:
                    current_route = new_route
                    current_distance = new_distance
                    
            if current_distance < best_distance:
                best_route = current_route
                best_distance = current_distance
            
            # print(f"Iteration {k}, Temperature: {temp:.4f}, Current Distance: {current_distance}, Best Distance: {best_distance}")
        
        return best_route, best_distance

# Parameters
initial_temp = 1000
cooling_rate = 0.995
max_iterations = 1000

# Running the simulated annealing algorithm
num_stations = 10
min_max_distance = (1, 100)
annealing_tsp = TSP(num_stations, min_max_distance)

best_route, best_distance = annealing_tsp.simulated_annealing(max_iterations)

# TODO: Add some stats about the best route and such based on the distances list

print("Best route found:", best_route)
print("Distance of the best route:", best_distance)
